# Dinámica de un Plasma

El plasma que se modela acontinuación corresponde a un conjunto de párticulas con:
* Masa 
* Cargadas 
* Puntuales condenadas a moverse en sobre una linea 

Las cuales sufren la interacción Coulombina debida sus vecinos, así como una sola fuerza de tipo mágnetica debido a un campo mágnetico externo perpendicular al espacio donde se mueven las partículas, he aquí:
* **La primera aproximación: Despreciamos el campo magnético generado por la carga en movimiento de las párticulas.**

Resultado que se alcanza al despreciarlo frente al campo externo, a esta aproximación se le suele llamar problema cuasi-estático.

Hasta ahora dadas las condiciones planteadas el modelo se realiza sobre un arreglo de 6 columnas y Np renglones:

$$ \begin{pmatrix}q_1 & m_1 & x_1 & vx_1 & vy_1 & vz_1\\ q_2 & m_2 & x_2 & vx_2 & vy_2 & vz_2\\ \vdots & \vdots & \vdots & \vdots & \vdots & \vdots\\ q_N & m_N & x_N & vx_N & vy_N & vz_n   \end{pmatrix} $$

In [18]:
#Damos el tamaño de la linea.
L = 3
#También el número de particulas.
Np = 2
#Ahora declaramos la cantidad de puntos en la red que describe el sistema.
Ng = 1001
#Con Intervalos de longitud determinada por:
dx = L/(Ng - 1)
Pi = float(pi);

In [2]:
#Creamos las particulas
partículas = zeros(Np, 6);

La dinámica de un plasma depende de la interacción de las particulas con el campo que genera la propia distribución de partículas, por esta razón necesitamos modelar también la distribución de las párticulas, así como los valores del campo eléctrico (aproximación Electroestática) en el espacio de confinamiento.

Para ello utilizamos modelos, comenzando por el cálculo de la distribució a partir de:

* ** Segunda aproximación: la distribución de partículas esta determinada por un modelo que consiste en una función de densidad que depende de la posición en el espacio de confinamiento. A esta aproximación se le suele llamar medio continuo.  **

Otro aspecto a tomar en cuneta es el confinamiento de las párticulas que tiene la cualidad de ser periódico, es decir que en los extremos de la linea las condiciones son las mismas, pues es necesaria la periodicidad para calcular el potencial vía **Transformada de Fourier**.

* **La Tercera aproximación: El plasma que trabajamos cumple tener el mismo valor de densidad de carga en las fronteras, en la realidad esta propiedad se puede aproximar por un plasma confinado a un "Círculo bastante grande" (lo suficiente para garantizar una densidad de carga uniforme por cada intervalo).**

Esta información se guarda en un arreglo de estado de 3 columnas y (Ng + 1) renglones, pues solo se conoce información en la posiciones de la red:

$$ \begin{pmatrix}\rho_1 & \phi_1 & E_1 \\ \rho_2 & \phi_2 & E_2 \\ \vdots & \vdots & \vdots \\ \rho_{Ng+1} & \phi_{Ng+1} & E_{Ng+1} \end{pmatrix} $$

Donde por conveniencia se el último renglon y corresponde al mismo punto experimental que el primero, pero que debido a la forma de como calculamos la densidad de carga lineal es util introducirlo.

In [9]:
#Creamos los puntos del espacio de confinamiento
red_mundo = zeros(Ng+1, 3);

In [15]:
#Calculamos la función de densidad de carga "Triángulos".
function rho_triángulos()
    for n in 1:Np
        q, m, x, vx, vy, vz = partículas[n,:]
        #Calculamos el punto a la Izquierda de la partícula
        punto_izq = Int(floor(x/dx))
        #Calculamos el punto a la Derecha de la partícula
        punto_der = punto_izq + 1
        #Distancia de la partícula al punto de la Izquierda
        d_izq = x % dx
        #Distancia de la partícula al punto de la Derecha
        d_der = dx - d_izq
        #Ahora calculamos el peso que contribuye al lado Izquierdo
        red_mundo[punto_izq, 1] += q/dx * (1 - d_izq/dx)
        #Y la contribución de pesoa a la Derecha
        red_mundo[punto_der, 1] += q/dx * (1 - d_der/dx)
    end
    #E introducimos las condiciones a la frontera
    red_mundo[Ng + 1, 1] = red_mundo[1, 1]
    return red_mundo
end 

rho_triángulos (generic function with 1 method)

## ## Explicar teoría física para cálculo de potencial, FFT y IFFT

In [21]:
#Calculemos el Potencial por via Transformada rápida de Fourier
function potencial_phi()
    #El primer paso consiste en 
    rhok = fft(red_mundo[1:Ng, 1])
    
    #Ahora obtenemos el potencial de esta función de densidad resolviendo la ecuación de Poison.
    #Recordando que si la carga neta es distinta de cero las condiciones a la frontera no se cumplen.
    phik = rhok
    phik[1] = 0
    
    #Notando que por estar en el sistema CGS se introduce un factor de 4Pi
    
    for k in 1:Ng-1
        phik[k+1] = 4 * Pi * phik[k+1]/(k * 2 * Pi * 1/L * sinc(k/Ng))^2
    end
    
    #Y podemos regresar al espacio de posiciones mediante la Transformada Inversa de Fourier.
    red_mundo[1:Ng, 2] = real(ifft(phik))
    
    #Finalmente introducimos la condición de continuidad en la frontera
    red_mundo[Ng + 1, 2] = red_mundo[1, 2]
    return
end     

potencial_phi (generic function with 1 method)

## ## Incluir una explicación sobre el cálculo del campo a partir del potencial, asi como la necesidad de calcular una interpolación del campo.

In [5]:
#Calculemos ahora el campo eléctrico, via diferencias finitas
function campo_eléctrico()
    for n in  2:Ng
        red_mundo[n, 3] = (red_mundo[n-1, 2]- red_mundo[n+1, 2])/(2 * dx)
    end
    
    #Declaramos los valores para los puntos faltantes
    red_mundo[1, 3] = (red_mundo[Ng, 2] - red_mundo[2, 2])/(2 * dx)
    red_mundo[Ng + 1, 3] = red_mundo[1, 3]
end

campo_eléctrico (generic function with 1 method)

Para poder conocer el campo en cada posición de cada partícula se introduce:

* ** La Cuarta aproxicimación: conocemos el campo en cada uno de los puntos sobre la red, sin embargo las posiciones de las partículas son arbitrarias, por lo tanto para conocer el campo aplicado sobre cada particula interpolamos los valores entre cada punto de la red. **

In [6]:
#Interpolación del Campo eléctrico
function interpolación_campo(x)
    #Encontramos el intervalo de puntos de la red que rondean la posición x
    N = x/dx + 1
    red_izq = Int(floor(N))
    red_der = red_izq + 1
    
    #Cerramos las condiciones en los extremos
    if red_izq == Ng
        red_der = 2
    end
    
    #Encontramos el desplazamiento entre el punto Izquierdo y Derecho
    d_L = x - dx * (red_izq - 1)
    
    #Caculamos el valor interpolado fuera de la red
    return red_mundo[red_izq, 3] + d_L/dx * (red_mundo[red_der, 3] - red_mundo[red_izq, 3])
end         

interpolación_campo (generic function with 1 method)

## Evolución temporal

La evolución de la partícula se obtiene a partir del calculo de las posiciones y velocidades de una partícula en cada punto de una serie de tiempos, espaciada uniformemente por un intervalo de tamaño ** dt **.

Pongamos un alto en este punto, y demos espacio a pensar cuales son las posibilidades para calcular la evolución temporal de forma númerica:

* Primeramente tenemos Np vectores a los que llamamos partículas, los cuales contienen la información de cada elemento del plasma, entre estos datos tenemos la posición y velocidades actuales de la partícula, y a partir de ellos, las ecuaciones de Maxwell y de Newton debemos obtener las posiciones y velocidades en cada elemento de la serie de tiempo para cada una de las párticulas.

* Tenemos varias opciones para integrar las velocidades y las posiciones: Métodojn de Euler, 

In [ ]:
#Calculamos las nuevas posiciones, ocupando la hipotesis de tratar con un problema cuasi-estático


Esto se traduce en calcular la aceleración de cada partícula, para rango de tiempos: 

Para calcular la evolución temporal del sistema se recurre a un método llamado ** " Leapfrog " **, que consiste:
* **La Quinta aproximación: introduce un retardo temporal en la velocidad, de tamaño: $$\cfrac{dt}{2}$$**

## Cálculos de Transformada de Fourier para un espacio de una dimensión

In [2]:
using PyPlot

INFO: Loading help data...


In [15]:
#Una transformada necesaria es la transformada de un operador derivada.
function difk(arr)
    for k in 0:length(arr)-1 
        arr[k+1] = arr[k+1] * im * k * (2 Pi/L) * sinc(2 k/N)
    end
    return arr
end

LoadError: syntax: missing separator in tuple
while loading In[15], in expression starting on line 4

In [16]:
#Otro operador importante es la segunda derivada, cuya transformada discreta es:
function ddifk(arr)
    for k in 0:length(arr)-1
        arr[k+1] = -arr[k+1] * (k * (2 Pi/L) * sinc(2 k/N))^2
        if k > length(arr)-1
            arr[k+1] = 0
        end
    end
    return arr
end

LoadError: syntax: missing separator in tuple
while loading In[16], in expression starting on line 4

In [19]:
#El operador de integración tambien es necesario y se obtine por:
function intk(arr)
    #El termino constante se introduce de una forma diferente
    a0 = arr[1]
    
    arr[1] = 0
    for k in 1:length(arr)-1
        arr[k+1] = arr[k+1] / (im * k * (2 Pi/L) * sinc(2 k/N)) + (im * a0 * L/(Pi*k))
    end
end

LoadError: syntax: missing separator in tuple
while loading In[19], in expression starting on line 8

In [18]:
#El operador de segunda integral se obtiene por:
function iintk(arr)
    #Introducimos el termino constante artificialmente
    a0 = arr[1]
    
    arr[1] = 0
    for k in 1:length(arr)-1
        arr[k+1] = -arr[k+1] / (k * (2 Pi/L) * sinc(2 k/N))^2 + (im * a0 * L/(pi*k))^2
    end
end

LoadError: syntax: missing separator in tuple
while loading In[18], in expression starting on line 8